# 08 - Carbon-Based Materials

**Overview** 

This notebook guides you through the simulation of ideal periodic crystalline structures of carbon-based materials. The goal of the calculations is to describe the electronic states (bands) of different phases and to understand how these states define their physical properties related to electronic conduction (insulators vs conductor vs semi metal) and mechanical properties (equilibrium lattice constant/volume and bulk modulus).

**Modules Setup**

NOTE: the following cell will return an error when run through Collab. This is because qepy requires an older version of numpy with respect to the one installed by default on Collab. After the error, Collab will ask you to restart the session. Click on "Restart session" and run the cell again. The second time the cell should run without returing an error.

In [ ]:
# @title Install qepy package  { display-mode: "form" }
!pip install qepy 

In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
!pip install -q ase > /dev/null
from ase.build import bulk, graphene, stack
from ase.io import write
from ase import Atoms
from ase.visualize import view
from qepy.calculator import QEpyCalculator
import time

**Problem** 

We want to understand how the crystal structure affects the physical properties of different phases of carbon-based materials. Can first principles calculations describe the electronic conduction and mechanical properties of different phases of carbon? What density functional is the best at reproducing experimental results?

**Model**
We will use different flavors of density functional theory (DFT) using a plane-wave (PW) basis set and pseudopotentials (PP) to describe core electrons. We will explore the main numerical parameters of PW-PP-DFT to make sure they do not significantly affect the final results. 



### Part 1: Convergence of Total Energy
**Questions**

Before you run any simulation, answer the following question(s):

1. For which phase of carbon do you expect to find a finite band gap (a energy gap between the highest occupied electronic states and the lowest unoccupied states)? 

Pick one phase of carbon and one flavor od density functional (PBE, LDA, or PBESol). Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

2. Explore the effect of the basis set size (ecutwfc) on the computed total energy of the system. 
3. Since in most situations we will be using a special type of PP, called ultrasoft (us or paw), we can get away with a smaller number of plane waves for the description of the orbitals. However, to describe the total electronic density (used by some of the functionals), we still need a larger number of plane waves. Thus, the code has a second energy parameters (ecutrho) that may affect the accuracy of the calculation. The general rule of thumb is to choose ecutrho to be 8-10 times larger than ecutwfc. 
4. How much do the two cutoffs affect your calculation time? 
5. The other parameter that may affect the accuracy and cost of the calculation is the threshold for the convergence of the self consistent cycle (SCF) of KS-DFT. This parameter (in QE called conv_thr) is size extensive (so for larger systems you can get away with larger values). 
6. For some systems, especially if metallic or with a small bandgap, SCF convergence may be helped by allowing the orbitals to be partially occupied, as if the system was at high temperature. This trick involves using a occupation of the band that is smeared (i.e. setting the 'occupations' keyword to 'smearing'). While for the systems in this notebook it may not have a big effect, check if adding the smearing improves the convergence of the calculation, i.e. by reducing the number of SCF steps. 
7. Since all the calculations rely on a small periodic cell, we should make sure that we repeat the calculations for different values of the Brillouin-zone wavevector. The resolution of the points in reciprocal space has a strong effect on the computed bands and total energy. Verify the convergence of the calculations with k-points sampling.

In [ ]:
# @title Simulation Parameters  { display-mode: "form" }
ecutwfc = 20  # @param {type:"number"}
ecutrho = 200  # @param {type:"number"}
conv_thr = 1e-6  # @param {type:"number"}
functional = "PBE"  # @param ["LDA", "PBE", "PBESOL", "BLYP"]
phase = "graphite"  # @param ["diamond", "graphite", "graphene"]
occupations = "fixed"  # @param ["smearing", "fixed"]
kpoints = "1 1 1 0 0 0"  # @param {type:"string"}

if functional == "LDA":
    pseudopotential = "C.pz-n-rrkjus_psl.0.1.UPF"
elif functional == "PBE":
    pseudopotential = "C.pbe-n-rrkjus_psl.1.0.0.UPF"
elif functional == "PBESOL":
    pseudopotential = "C.pbesol-n-rrkjus_psl.1.0.0.UPF"

qe_options = {
    '&control': {
        'calculation': "'scf'",
        'prefix': "'"+phase+"'",
        'pseudo_dir': "'/tmp/'",
    },
    '&system': {
        'ibrav': 0,
        'ecutwfc': ecutwfc,
        'ecutrho': ecutrho,
        'occupations': "'"+occupations+"'",
        'smearing': "'gaussian'",
        'degauss': 0.01,
    },
    '&electrons': {
        'conv_thr': conv_thr,
    },
    'atomic_species': ['C 28.0855 '+pseudopotential],
    'k_points automatic': [kpoints],
}

# Download pseudopotential
!wget -O /tmp/$pseudopotential http://pseudopotentials.quantum-espresso.org/upf_files/$pseudopotential 
# In case wget is not available
# !curl -l http://pseudopotentials.quantum-espresso.org/upf_files/$pseudopotential -o ./$pseudopotential

In [ ]:
# @title Generate and Visualize the System  { display-mode: "form" }
show = 'reciprocal'  # @param ["direct", "reciprocal"]

if phase == "graphite":
    a = 2.46   # in-plane lattice parameter (Å)
    c = 6.70   # c-axis (Å), ~ 2 × 3.35 Å interlayer spacing

    # Hexagonal cell vectors
    cell = [
        (a, 0.0, 0.0),
        (a/2, a*np.sqrt(3)/2, 0.0),
        (0.0, 0.0, c)
    ]

    # AB stacking (P6_3/mmc): 4 atoms per conventional hex cell
    # fractional coords: layer 1 (z=0):        (0,0,0), (1/3,1/3,0)
    #                     layer 2 (z=1/2):     (0,0,1/2), (2/3,2/3,1/2)
    frac = [
        (0.0,   0.0,   0.0),
        (1/3,   1/3,   0.0),
        (0.0,   0.0,   0.5),
        (2/3,   2/3,   0.5),
    ]
    system = Atoms('C4', cell=cell, scaled_positions=frac, pbc=True)
elif phase == "graphene":
    # Conventional hexagonal cell
    a = 2.46
    system = graphene(formula='C2', a=a, vacuum=10.0)  # ~10 Å vacuum
elif phase == "diamond":
    # Conventional diamond cubic cell
    a = 3.567
    system = bulk('C', 'diamond', a=a)
calc = QEpyCalculator(qe_options=qe_options, logfile='C_diamond.out')
system.calc=calc
    
if show == 'direct':
    view(system, viewer='x3d')
    write('-', system, format='extxyz')
elif show == 'reciprocal':
    lat = system.cell.get_bravais_lattice()
    lat.plot_bz(show=True)
    print(lat.description())

In [ ]:
# @title Run the calculation { display-mode: "form" }
start = time.time()
energy = system.get_potential_energy()
efermi = calc.get_fermi_level()
end = time.time()
elapsed = end - start
print(f"QE calculation time: {elapsed/60:.2f} minutes ({elapsed:.1f} seconds)")
print(f"Total energy: {energy:.6f} eV")
print(f"Fermi energy: {efermi:.6f} eV")

# run a non-scf calculation to get band energies, increase k-points density
qe_options['k_points automatic']= ['8 8 8 0 0 0']
energies, dos = system.calc.get_dos(qe_options, width=0.6)

# run band structure calculation along high-symmetry path
if phase == "graphene":
    path = system.cell.bandpath('GKMG',npoints=61)
    qe_options['&system']['nbnd']=6
elif phase == "graphite":
    path = system.cell.bandpath('GMKGALHA',npoints=61)
    qe_options['&system']['nbnd']=12
elif phase == "diamond":
    path = system.cell.bandpath('GXWKGLUWLK',npoints=61)
    qe_options['&system']['nbnd']=6

band = system.calc.get_band_structure(qe_options, kpts=path, reference=efermi)

bp=band.plot()


### Part 2: Equation of State

While the calculations above give you a reasonable starting point on the selection of the parameters, often there are still strong variations in the total energy of the system even for very tight values of the parameters. In reality, most of the applications of DFT only care about relative energies. For example, in a chemical reaction we often want to compare the energy of the products with the one of the reactants. For phase stability we often only care about relative energies of different phases. When looking at energy differences, it is often the case that error cancellation speed up the convergence of the results with the numerical parameters. In the following, we will explore the effect of changing the lattice spacing of the crystals on the total energy, so as to determine the equilibrium lattice constant and the corresponding bulk modulus. 

Before you run any simulation, answer the following question(s):

1. Which phase of carbon do you expect to have the highest stiffness? 

Pick one phase of carbon. Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

2. Run a series of DFT calculations while changing the lattice parameter of the system. Visualize the energy vs. lattice constant (a.k.a. the equation of state or EoS) and identify the equilibrium lattice parameter and the curvature of the EoS (the second derivative at the minimum). 
3. How much do the numerical parameters of the simulation affect the results above?
4. Perform simulations for the different phases (make sure to keep the numerical parameters consistent across the different simulations) and plot the EoS results into a single plot. Which phase is the most stable? Could you use these plots to predict the transition pressure to convert graphite into diamond? Compare the results with the experimental litterature. 

**Homework Assignment**

Pick one (or more) of the following projects:
1. Select a periodic molecular or crystalline system and perform DFT simulations with at least two different density functionals (e.g. LDA and PBE)
2. Optimize the lattice parameter of the system (equation of state)
3. Verify the effect of the size of the basis set on the optimal lattice parameter
